In [2]:
from pyserini.search import SimpleSearcher
import json
from iirc_query_decomposition import decompose

searcher = SimpleSearcher('./data/iirc_index')

In [4]:
test_set = json.load(open("./data/iirc_test.json",'r'))

In [7]:
all_q = []

for item in test_set:
    for q in item['questions']:
        q['text'] = item['text']
        q['title'] = item['title']
        q['links'] = item['links']
        all_q.append(q)

In [10]:
import json
import numpy as np
from tqdm import tqdm

ONLY_LINKS = True
limit_by_query= 1000
new_test = []

for q in tqdm(all_q):
    decomposition = decompose(q['question'])
    q['decomposition'] = decomposition
    titles = [l['target'].lower() for l in q['links']] + [q['title'].lower()]
    decs = []
    
    for d in decomposition:
        hits = searcher.search(d,k=limit_by_query)
        chosen = []
        for hit in hits:
            hit = json.loads(hit.raw)
            if ONLY_LINKS:
                if hit['title'].lower() in titles:
                    chosen.append(hit)
            else:
                chosen.append(hit)
        decs.append({
            "question": d,
            "documents": chosen
        })
    q['decomposition'] = decs

    new_test.append(q)


100%|██████████| 2/2 [00:04<00:00,  2.10s/it]


In [11]:
json.dump(new_test, open("data/to_rerank.json",'w'))